In [68]:
import math

In [169]:
def long_divide(a,b):
	q = a//b
	adiffr0 = a - q*b
	adiff0 = abs(adiffr0)
	adiffr1 = adiffr0 - b
	adiff1 = abs(adiffr1)
	if adiff0 < adiff1:
		return q,adiffr0
	else:
		return q+1,adiffr1

def gcd(a,b,gcd_only=True):
	carda = (1,0)
	cardb = (0,1)

	q,r = long_divide(a,b)
	cardc = (carda[0] - (q*cardb[0]),carda[1] - (q*cardb[1]))
	carda = cardb
	cardb = cardc
	a = b
	b = r

	while r != 0:
		q, r = long_divide(a, b)
		cardc = (carda[0]-(q*cardb[0]), carda[1]-(q*cardb[1]))
		carda = cardb
		cardb = cardc
		a = b
		b = r

	if a < 0:
		a = -a
		carda = (-carda[0],-carda[1])

	if gcd_only:
		return a
	else:
		return a,carda
    
def modi(a,n):
	if a < 0:
		return n*(-(a // n)) + a
	return a % n

In [233]:
#TODO update this stuff so it actually matches the p-adic definition
def tpat_inner(p,max_depth,pn=None,pd=None,depth=0,print_as=None):
    if print_as is None:
        print_as = 'frac'
    s = ''
    if depth > max_depth:
        return s
    if (pn is None):
        pn = 1
        pd = 1
        
        
    
    #add myself to s first
    s += '\t'*depth
    #preprint the first node (we need this still so that the first level is right)
    if depth != 0:
        s += '['
        if print_as == 'dec':
            s += '.{:.3f}\n'.format(pn/pd)
        elif print_as == 'frac':
            s += '.{}/{}\n'.format(pn,pd)
        elif print_as == 'exp':
            s += '.({},{})\n'.format(pn,depth)
        elif  print_as == 'hex-exp':
            s += '.({},{})\n'.format(hex(pn)[2:],hex(depth)[2:])
    
    
    #add on my children
    #this is where the actual math is
    d = pd*p
    i = pn//p
    r = (pn % p) - 1
    lown = p*r + r + 1 + (p**2)*i
    to_add = p if depth != 0 else p-1
    n = lown
    while to_add > 0:
        if ((n % p) != 0):
            s += tpat_inner(p,max_depth,pn=n,pd=d,depth=depth+1,print_as=print_as)
            to_add -= 1
        n += 1
    
    #close the parens
    s += '\t'*depth
    s += ']\n'
    
    return s

def tex_p_adic_tree(p,max_depth,print_as=None):
    if print_as is None:
        print_as = 'frac'#options are dec, frac, exp, hex-exp
    s = ''
    s += '\\Tree[.1'
    s += tpat_inner(p,max_depth,print_as=print_as)
#     s = s.replace('(','{').replace(')','}')
    print(s)

In [235]:
tex_p_adic_tree(3,4,print_as='frac')

\Tree[.1	[.1/3
		[.1/9
			[.1/27
				[.1/81
				]
				[.2/81
				]
				[.4/81
				]
			]
			[.2/27
				[.5/81
				]
				[.7/81
				]
				[.8/81
				]
			]
			[.4/27
				[.10/81
				]
				[.11/81
				]
				[.13/81
				]
			]
		]
		[.2/9
			[.5/27
				[.14/81
				]
				[.16/81
				]
				[.17/81
				]
			]
			[.7/27
				[.19/81
				]
				[.20/81
				]
				[.22/81
				]
			]
			[.8/27
				[.23/81
				]
				[.25/81
				]
				[.26/81
				]
			]
		]
		[.4/9
			[.10/27
				[.28/81
				]
				[.29/81
				]
				[.31/81
				]
			]
			[.11/27
				[.32/81
				]
				[.34/81
				]
				[.35/81
				]
			]
			[.13/27
				[.37/81
				]
				[.38/81
				]
				[.40/81
				]
			]
		]
	]
	[.2/3
		[.5/9
			[.14/27
				[.41/81
				]
				[.43/81
				]
				[.44/81
				]
			]
			[.16/27
				[.46/81
				]
				[.47/81
				]
				[.49/81
				]
			]
			[.17/27
				[.50/81
				]
				[.52/81
				]
				[.53/81
				]
			]
		]
		[.7/9
			[.19/27
				[.55/81
				]
				[.56/81
				]
				[.58/81
				]
			]
			[.20/27
				[.59/8

In [1]:
class p_adic:
    
    def __init__(self,p,a,n,reduce=True,print_as='exp'):
        if reduce and (a == 0):
            n = 0#zero should always look like this
        while reduce and (a != 0) and ((a % p) == 0):
            a //= p
            n -= 1
        
        self.p = p
        self.a = a
        self.n = n
        
        self.print_as = print_as
        
        self.denom = p**n
        
        self.par = None
        self.chld = None
    
    def __repr__(self):
        if self.n == 0:
            return str(self.a)
        if self.print_as == 'exp':
            return "{}*{}^({})".format(self.a,self.p,-self.n)
        elif self.print_as == 'frac':
            return "{}/{}".format(self.a,self.denom)
    
    def s(self,pm):
#         if (self.a == 1) and (self.n == 0):#make 1 act like 1/2
#             numerator = 0 if pm == -1 else 1
#             return p_adic(self.p,numerator,0,print_as=self.print_as)
        if (pm == -1) and (self.a <= 1):
            return p_adic(self.p,-self.a,self.n,print_as=self.print_as)
        if (pm == 1) and (self.a >= (self.denom-1)):
            return p_adic(self.p,self.denom+self.a,self.n,print_as=self.print_as)#1/p^0 = 1/1 = 1
        
        if modi(self.a,self.p) != modi(-pm,self.p):
            return p_adic(self.p,self.a + pm,self.n,print_as=self.print_as)
        else:
            return p_adic(self.p,self.a + (2*pm),self.n,reduce=False,print_as=self.print_as)
    
    def is_ancestor_of(self,other):
        if other.p != self.p:
            raise TypeError("{} and {} are not in the same ring")
        if other.n < self.n:
            #this case is necessary to handle the "1/3 is within the bounds of 2/9" case
            return False
        if (other.n == self.n) and (other.a == self.a):
            return True#I am my own ancestor
        
        cousin_prev = self.s(-1)
        cousin_next = self.s(1)
        
        over_two_scale = self.p**(other.n - self.n)
        mo = (cousin_prev.a + self.a)*over_two_scale
        po = (cousin_next.a + self.a)*over_two_scale
        cmp = other.a*2#double this instead of halving the cousins to keep things in ints
        
        return (mo < cmp) and (cmp < po)
        
    
    def __lt__(self,other):
        diff = self - other
        return diff.a < 0
    
    def __le__(self,other):
        return (self == other) or (self < other)
    
    def __ge__(self,other):
        return not (self < other)
    
    def __eq__(self,other):
        return (self.p == other.p) and (self.a == other.a) and (self.n == other.n)
    
    def __gt__(self,other):
        return (self >= other) and (not (self == other))
    
    def __add__(self,other):
        if other.p != self.p:
            raise TypeError("{} and {} are not in the same ring")
        if self.n ==  other.n:
            return p_adic(self.p,self.a+other.a,self.n,print_as=self.print_as)
        elif self.n < other.n:
            return p_adic(self.p,((self.p**(other.n - self.n))*self.a)+other.a,
                          other.n,print_as=self.print_as)
        else:
            return p_adic(self.p,self.a+(other.a*(self.p**(self.n - other.n))),
                          self.n,print_as=self.print_as)
    
    def __abs__(self):
        return p_adic(self.p,abs(self.a),self.n,print_as=self.print_as)
    
    def __neg__(self):
        return p_adic(self.p,-self.a,self.n,print_as=self.print_as)
    
    def __sub__(self,other):
        return self + (-other)
    
    def __mul__(self,other):
        return p_adic(self.p,self.a*other.a,self.n+other.n,print_as=self.print_as)
    
    def parent(self):
        if self.par is not None:
            return self.par
        opt1 = (self.a - (self.a % self.p))//self.p#go down
        if (opt1 % self.p) != 0:
            return p_adic(self.p,opt1,self.n-1,print_as=self.print_as)
        opt2 = (self.a + (self.p - (self.a % self.p)))#go up
        self.par = p_adic(self.p,opt2//self.p,self.n-1,print_as=self.print_as)
        return self.par
    
    def children(self):
        if self.chld is not None:
            return self.chld
        cousin_prev = self.s(-1)
        ch_denom = self.n+1
        ch1_numerator = int(math.ceil((cousin_prev.a + self.a)*self.p/2))
        #just brute-force the actual start since it's only like a O(p) calculation
        ch1 = p_adic(self.p,ch1_numerator,ch_denom,print_as=self.print_as)
        ch1inc = p_adic(self.p,2*ch1_numerator,ch_denom,print_as=self.print_as)
        double_midpoint = self + cousin_prev
        while ch1inc <= double_midpoint:
            ch1_numerator += 1
            ch1 = p_adic(self.p,ch1_numerator,ch_denom,print_as=self.print_as)
            ch1inc = p_adic(self.p,2*ch1_numerator,ch_denom,print_as=self.print_as)
        children = [ch1]
        ch_num = ch1_numerator + 1
        ch_inc = ch_num*2
        double_upper_midpoint = self.p*(self.a + self.s(1).a)
        ch_denom_val = self.p**(ch_denom)
        while (ch_num < ch_denom_val) and (ch_inc < double_upper_midpoint):
            if (ch_num % self.p) != 0:
                children.append(p_adic(self.p,ch_num,ch_denom,print_as=self.print_as))
            ch_num += 1
            ch_inc += 2
        
        self.chld = children
        return self.chld
    
    def greedy_search(self,target):
#         print(self)
        if self == target:
#             print("WE HAVE ARRIVED")
            return True
        
        go_down = self.is_ancestor_of(target)
        
        if not go_down:
            #then go up!
            return (self.parent()).greedy_search(target)
        
        #otherwise it must be down!
        goto_ch = None
        goto_dist = p_adic(self.p,1,0,print_as=self.print_as)#biggest possible distance is 1
        for ch in self.children():
            dist_this = abs(ch - target)
            if dist_this < goto_dist:
                goto_ch = ch
                goto_dist = dist_this
        
        return goto_ch.greedy_search(target)

In [360]:
y = p_adic(5,(5**2)-1,2,print_as='frac')
x = p_adic(3,1,0,print_as='frac')
print(x,x.children())

1 [1/3, 2/3]


In [225]:
x.children()

[0/3, 1/3]

In [158]:
x.s(1)

4/9

In [198]:
x = p_adic(3,26,4,print_as='frac')
y = p_adic(3,28,4,print_as='frac')
x,y

(26/81, 28/81)

In [199]:
x.greedy_search(y)

26/81
8/27
2/9
1/3
4/9
10/27
28/81
WE HAVE ARRIVED


True

In [227]:
x = p_adic(3,40,4,print_as='frac')
y = p_adic(3,41,4,print_as='frac')
x,y

(40/81, 41/81)

In [228]:
x.greedy_search(y)

40/81
13/27
4/9
1/3
1
2/3
5/9
14/27
41/81
WE HAVE ARRIVED


True

In [362]:
p = 7
n = 1
x = p_adic(p,1,0,print_as='frac')
y = p_adic(p,1,n,print_as='frac')
x,y

(1, 1/7)

In [363]:
x.children()

[1/7, 2/7, 3/7, 4/7, 5/7, 6/7]

In [364]:
x.greedy_search(y)

1
1/7
WE HAVE ARRIVED


True

In [321]:
x = p_adic(5,1562,5,print_as='frac')
y = p_adic(5,1563,5,print_as='frac')
x,y

(1562/3125, 1563/3125)

In [322]:
x.greedy_search(y)

1562/3125
312/625
62/125
12/25
2/5
1
3/5
13/25
63/125
313/625
1563/3125
WE HAVE ARRIVED


True

More generally, the worst (i.e. going through 1) split is:

In [323]:
p = 17
n = 10
midp = (p**n) >> 1
x = p_adic(p,midp,n,print_as='frac')
y = p_adic(p,midp+1,n,print_as='frac')
x,y

(1007996950224/2015993900449, 1007996950225/2015993900449)

In [324]:
x.greedy_search(y)

1007996950224/2015993900449
59293938248/118587876497
3487878720/6975757441
205169336/410338673
12068784/24137569
709928/1419857
41760/83521
2456/4913
144/289
8/17
1
9/17
145/289
2457/4913
41761/83521
709929/1419857
12068785/24137569
205169337/410338673
3487878721/6975757441
59293938249/118587876497
1007996950225/2015993900449
WE HAVE ARRIVED


True

OR

In [325]:
p = 17
n = 10
x = p_adic(p,1,n,print_as='frac')
y = p_adic(p,(p**n)-1,n,print_as='frac')
x,y

(1/2015993900449, 2015993900448/2015993900449)

In [326]:
x.greedy_search(y)

1/2015993900449
1/118587876497
1/6975757441
1/410338673
1/24137569
1/1419857
1/83521
1/4913
1/289
1/17
1
16/17
288/289
4912/4913
83520/83521
1419856/1419857
24137568/24137569
410338672/410338673
6975757440/6975757441
118587876496/118587876497
2015993900448/2015993900449
WE HAVE ARRIVED


True

Let's do an all-pairs trial of this

In [368]:
p = 7
max_n = 4#7^8 pairs, so ~5 million checks
for n1 in range(max_n):
    denom1 = p**n1    
    maxnum1 = (denom1 - 1) if n1 != 0 else 1
    for num1 in range(1,maxnum1+1):
        x_1 = p_adic(p,num1,n1)
        for n2 in range(max_n):
            denom2 = p**n2
            maxnum2 = (denom2 - 1) if n2 != 0 else 1
            for num2 in range(1,maxnum2+1):
                x_2 = p_adic(p,num2,n2)
#                 print(x_1,x_2)
                assert(x_1.greedy_search(x_2))

# YEET